In [ ]:
#importing necessary variables
import numpy as np
import numpy.linalg as npl

c=3*10**8
ns=250        
par=2          
snb=31          

sigo=0.01        
sigh=0.01          
ar=np.empty([ns,par+1])    
z=[]                       
M=[]                        
accn=0                      


Cov=open('jla_mub_covmatrix.txt','r')
data=open('jla_mub_0.txt','r')                                  

l1=1
C=np.loadtxt(Cov)
C=np.reshape(C,(snb,snb))
cinv=npl.inv(C)          


z,M=np.loadtxt(data, usecols=[0,1])

#defining necessary functions

def eta(a,om):                     
    if om>=0.9999:
        om=0.9999
    elif om<=0.0:
        om=0.000001
    else:
        pass
    s=((1.0-om)/om)**(1.0/3.0)
    n= 2.0*(np.sqrt((s*s*s)+1))*(((a**(-4.0))-(0.1540*s*(a**(-3.0)))+(0.4304*s*s*(a**(-2.0)))+(0.19097*s*s*s*(a**(-1.0)))+(0.066941*s*s*s*s)))**(-1.0/8.0)
    return n

def Dl(z,om):                      
    eta1=eta(1,om)
    eta2=eta(1/(1+z),om)
    d=(3000.0*(1+z))*(eta1-eta2)
    return d

def mu(z,om,h):                                   
    d=Dl(z,om)
    m=25-(5*np.log10(h))+(5*np.log10(d))
    return m

diff=np.empty(snb)  
def likelihood(om,h,z,M):                 
    if om<=0.0 or h<=0.0:              
        l=-1.e100
    else:
        for i in range(snb):
            diff[i]=M[i]-mu(z[i],om,h)
        dt=np.dot(cinv,diff)
        l=-0.5*np.dot(np.transpose(diff),dt)     
    return l    #l is ln(L)

#gelman rubin test
nchain=50
omchainm=[]
omchainvar=[]
hchainmean=[]
hchainvar=[]

for chain in range(nchain):
    print("Progress: "+str((chain*100)/nchain))
    ar[0,0]=np.random.uniform()
    ar[0,1]=np.random.uniform()
    ar[0,2]=likelihood(ar[0,0], ar[0,1], z, M)
    
    for i in range(1,ns):
        lpre=ar[i-1,2]
        omnext=np.random.normal(ar[i-1,0],sigo)
        hnext=np.random.normal(ar[i-1,1],sigh)
        if omnext>0.9999:
            omnext=0.999
        lnext=likelihood(omnext, hnext, z, M)
            
        if lnext>=lpre:
            ar[i,0]=omnext
            ar[i,1]=hnext
            ar[i,2]=lnext
            accn=accn+1
            print("Accepted with higher likelihood.")
        else:
            x=np.random.uniform()
            if (lnext-lpre)>np.log(x):
                ar[i,0]=omnext
                ar[i,1]=hnext
                ar[i,2]=lnext
                accn=accn+1
                print("Accepted with lesser likelihood.")
            else:
                ar[i,0]=ar[i-1,0]
                ar[i,1]=ar[i-1,1]
                ar[i,2]=lpre
                print("rejected")

#burn in
r=ns//25
omm=np.mean(ar[r:,0])
hm=np.mean(ar[r:,1])
omvar=np.var(ar[r:,0])
hvar=np.var(ar[r:,1])
omchainm.append(omm)
hchainmean.append(hm)
omchainvar.append(omvar)
hchainvar.append(hvar)

omsigchsq=np.mean(omchainvar)
omsigmeansq=np.var(omchainm)

ratioom=(((49.0/50.0)*omsigchsq)+(omsigmeansq/50.0))/omsigchsq

hsigchsq=np.mean(hchainvar)
hsigmeansq=np.var(hchainmean)

ratioh=((((nchain-1.0)/nchain)*hsigchsq)+(hsigmeansq/nchain))/hsigchsq

print("Value of Gelman Rubin convergence ratio for omega using "+str(ns)+" samples is: "+str(ratioom))
print("Value of Gelman Rubin convergence ratio for h using "+str(ns)+" samples is: "+str(ratioh)) 